### Newton's Method

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [2]:
def d1(PVF, K, disc, x, T):
    return np.log(PVF/(K*disc))/(x*np.sqrt(T))+x*np.sqrt(T)/2
            
def d2(PVF, K, disc, x, T):
    return np.log(PVF/(K*disc))/(x*np.sqrt(T))-x*np.sqrt(T)/2

In [3]:
def fc(PVF, K, disc, x, T, cm):
    return PVF*norm.cdf(d1(PVF, K, disc, x, T))-K*disc*norm.cdf(d2(PVF, K, disc, x, T))-cm
def fc_p(PVF, K, disc, x, T):
    return PVF*np.sqrt(T/(2*np.pi))*np.exp(-d1(PVF, K, disc, x, T)*d1(PVF, K, disc, x, T)/2)

In [4]:
def fp(PVF, K, disc, x, T, pm):
    return -PVF*norm.cdf(-d1(PVF, K, disc, x, T))+K*disc*norm.cdf(-d2(PVF, K, disc, x, T))-pm
def fp_p(PVF, K, disc, x, T):
    return PVF*np.sqrt(T/(2*np.pi))*np.exp(-d1(PVF, K, disc, x, T)*d1(PVF, K, disc, x, T)/2)

In [5]:
#present value of Foward price
PVF = 2360.174 
#discount factor
disc = 0.993
T = 139/252

In [6]:
strike_lst = [2150,2175,2200,2225,2250,2275,2300,2325,2350,2375,2400,2425,2450,2475,2500,2550,2600,2650,2700,2800]
cm_lst = [260,238.8500061,218.1499939,197.9499969,178.1499939,159,140.5500031,122.8499985,106.0500031,90.20000076,75.45000076,61.94999886,49.69999886,39.04999924,29.84999943,16.04999971,7.949999809,3.799999952,1.899999976,0.650000006]
pm_lst = [35.25,38.95000076,43,47.59999847,52.64999962,58.35000038,64.70000076,71.84999847,79.79999924,88.79999924,98.79999924,110.1500015,122.75,136.8500061,152.5,187.9499969,229.4499969,275.8999939,323.6500092,421.6000061]

In [7]:
def call_implied_vol(PVF, K, disc, T, cm):
    x_new = 0.25
    x_old = 1.25
    tol = 10**(-6)
    while (abs(x_new - x_old)>tol):
        x_old = x_new
        x_new = x_new - (fc(PVF, K, disc, x_new, T, cm))/fc_p(PVF, K, disc, x_new, T)
    return x_new

def put_implied_vol(PVF, K, disc, T, pm):
    x_new = 0.25
    x_old = 1.25
    tol = 10**(-6)
    while (abs(x_new - x_old)>tol):
        x_old = x_new
        x_new = x_new - (fp(PVF, K, disc, x_new, T, pm))/fp_p(PVF, K, disc, x_new, T)
    return x_new

In [8]:
#calculate call option implied vol
call_implied_vol_lst = [0] * len(strike_lst)
put_implied_vol_lst = [0] * len(strike_lst)

for i in range(len(strike_lst)):
    call_implied_vol_lst[i] = call_implied_vol(PVF, strike_lst[i], disc, T, cm_lst[i])
    put_implied_vol_lst[i] = put_implied_vol(PVF, strike_lst[i], disc, T, pm_lst[i])

In [9]:
df = pd.DataFrame(columns=["Spot Price", "Strike Price", "Implied Vol Call", "Implied Vol Put"])

In [10]:
df["Strike Price"] = strike_lst
df["Implied Vol Call"] = call_implied_vol_lst
df["Implied Vol Put"] = put_implied_vol_lst

In [11]:
df["Implied Vol Difference"] = df["Implied Vol Call"] - df["Implied Vol Put"]

In [12]:
df["Spot Price"] = 2381

In [13]:
df

,Spot Price,Strike Price,Implied Vol Call,Implied Vol Put,Implied Vol Difference
0,2381,2150,0.170840,0.171814,-0.000974
1,2381,2175,0.166065,0.167031,-0.000967
2,2381,2200,0.161341,0.162117,-0.000777
3,2381,2225,0.156654,0.157348,-0.000693
4,2381,2250,0.151734,0.152437,-0.000702
5,2381,2275,0.146901,0.147613,-0.000712
6,2381,2300,0.142104,0.142752,-0.000648
7,2381,2325,0.137290,0.137955,-0.000665
8,2381,2350,0.132544,0.133086,-0.000542
9,2381,2375,0.127793,0.128364,-0.000571


### An Explicit Implied Volatility Formula

In [14]:
PVF = 2360.174
disc = 0.993
T = 139/252

In [15]:
def A_(y):
    return (np.exp((1-2/np.pi)*y)-np.exp(-(1-2/np.pi)*y))**2

def B_(y, R):
    return 4*(np.exp(2*y/np.pi)+np.exp(-2*y/np.pi))-2*np.exp(-y)*(np.exp((1-2/np.pi)*y)+np.exp(-(1-2/np.pi)*y))*(np.exp(2*y)+1-R*R)

def C_(y, R):
    return np.exp(-2*y)*(R*R-(np.exp(y)-1)**2)*((np.exp(y)+1)**2-R*R)

In [16]:
# Polya: accurate approximation for N(x)
def cdf_A(x):
    if x>=0:
        return 1/2 + np.sqrt(1-np.exp(-2*x*x/np.pi))/2
    else:
        return 1/2 - np.sqrt(1-np.exp(-2*x*x/np.pi))/2

In [17]:
def call_implied_vol_explicit_formula(PVF, K, disc, T, cm):
    y = np.log(PVF/(K*disc))
    alpha_c = cm/(K*disc)
    R = 2*alpha_c - np.exp(y) + 1
    
    A = A_(y)
    B = B_(y, R)
    C = C_(y, R)
    
    beta = (2*C)/(B+np.sqrt(B*B+4*A*C))
    gamma = -(np.pi/2)*np.log(beta)
    
    if y >= 0:
        c0 = K*disc*(np.exp(y)*cdf_A(np.sqrt(2*y))-1/2)
        if cm <= c0:
            sigma = (np.sqrt(gamma+y)-np.sqrt(gamma-y))/np.sqrt(T)
        else:
            sigma = (np.sqrt(gamma+y)+np.sqrt(gamma-y))/np.sqrt(T)
    
    else:
        c0 = K*disc*(np.exp(y)/2-cdf_A(-np.sqrt(-2*y)))
        if cm <= c0:
            sigma = (-np.sqrt(gamma+y)+np.sqrt(gamma-y))/np.sqrt(T)
        else:
            sigma = (np.sqrt(gamma+y)+np.sqrt(gamma-y))/np.sqrt(T)
    
    return sigma

In [18]:
def put_implied_vol_explicit_formula(PVF, K, disc, T, pm):
    y = np.log(PVF/(K*disc))
    alpha_p = pm/(K*disc)
    R = 2*alpha_p + np.exp(y) - 1
    
    A = A_(y)
    B = B_(y, R)
    C = C_(y, R)
    
    beta = (2*C)/(B+np.sqrt(B*B+4*A*C))
    gamma = -(np.pi/2)*np.log(beta)
    
    if y >= 0:
        p0 = K*disc*(-np.exp(y)*cdf_A(-np.sqrt(2*y))+1/2)
        if pm <= p0:
            sigma = (np.sqrt(gamma+y)-np.sqrt(gamma-y))/np.sqrt(T)
        else:
            sigma = (np.sqrt(gamma+y)+np.sqrt(gamma-y))/np.sqrt(T)
    
    else:
        p0 = K*disc*(-np.exp(y)/2+cdf_A(np.sqrt(-2*y)))
        if pm <= p0:
            sigma = (-np.sqrt(gamma+y)+np.sqrt(gamma-y))/np.sqrt(T)
        else:
            sigma = (np.sqrt(gamma+y)+np.sqrt(gamma-y))/np.sqrt(T)
    
    return sigma

In [19]:
#calculate call option implied vol
call_implied_vol_lst = [0] * len(strike_lst)
put_implied_vol_lst = [0] * len(strike_lst)

for i in range(len(strike_lst)):
    call_implied_vol_lst[i] = call_implied_vol_explicit_formula(PVF, strike_lst[i], disc, T, cm_lst[i])
    put_implied_vol_lst[i] = put_implied_vol_explicit_formula(PVF, strike_lst[i], disc, T, pm_lst[i])

In [20]:
df = pd.DataFrame(columns=["Strike Price", "Implied Vol Call", "Implied Vol Put"])
df["Strike Price"] = strike_lst
df["Implied Vol Call"] = call_implied_vol_lst
df["Implied Vol Put"] = put_implied_vol_lst
df["Implied Vol Difference"] = df["Implied Vol Call"] - df["Implied Vol Put"]

In [21]:
df

,Strike Price,Implied Vol Call,Implied Vol Put,Implied Vol Difference
0,2150,0.168468,0.169455,-0.000987
1,2175,0.164146,0.165123,-0.000977
2,2200,0.159836,0.160619,-0.000784
3,2225,0.155521,0.156219,-0.000698
4,2250,0.150924,0.151630,-0.000706
5,2275,0.146366,0.147080,-0.000715
6,2300,0.141791,0.142440,-0.000649
7,2325,0.137143,0.137809,-0.000666
8,2350,0.132502,0.133044,-0.000542
9,2375,0.127791,0.128362,-0.000571
